In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
  # Install dependencies
  ! pip install --upgrade pip
  ! pip install czitools
  ! pip install ipyfilechooser

In [1]:
# import the required libraries
from czitools import metadata_tools as czimd
from czitools import misc_tools
from ipyfilechooser import FileChooser
from IPython.display import display, HTML
from pathlib import Path
import os
import requests

### Define Parameters for Data Loading

In [2]:
# Folder containing the input data
if IN_COLAB:
    INPUT_FOLDER = 'data/'
if not IN_COLAB:
    INPUT_FOLDER = '../../data/'

# Path to the data on GitHub
GITHUB_IMAGES_PATH = "https://raw.githubusercontent.com/sebi06/czitools/main/data.zip"

# Download data
if not (os.path.isdir(INPUT_FOLDER)):
    compressed_data = './data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_IMAGES_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

FileChooser(path='F:\Github\czitools\data', filename='', title='', show_hidden=False, select_desc='Select', ch…

In [ ]:
fc = FileChooser(INPUT_FOLDER)
fc.filter_pattern = '*.czi'
display(fc)

In [3]:
# Print the selected path, filename, or both
print(fc.selected_path)
print(fc.selected_filename)
print(fc.selected)

F:\Github\czitools\data
CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi
F:\Github\czitools\data\CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi


In [4]:
# get only specific metadata
czi_dimensions = czimd.CziDimensions(fc.selected)
print("SizeS: ", czi_dimensions.SizeS)
print("SizeT: ", czi_dimensions.SizeT)
print("SizeZ: ", czi_dimensions.SizeZ)
print("SizeC: ", czi_dimensions.SizeC)
print("SizeY: ", czi_dimensions.SizeY)
print("SizeX: ", czi_dimensions.SizeX)

SizeS:  None
SizeT:  3
SizeZ:  5
SizeC:  2
SizeY:  170
SizeX:  240


In [5]:
# and get more info
czi_scaling = czimd.CziScaling(fc.selected)
czi_channels = czimd.CziChannelInfo(fc.selected)
czi_bbox = czimd.CziBoundingBox(fc.selected)
czi_objectives = czimd.CziObjectives(fc.selected)
czi_detectors = czimd.CziDetector(fc.selected)
czi_microscope = czimd.CziMicroscope(fc.selected)
czi_sample = czimd.CziSampleInfo(fc.selected)

In [6]:
# get the complete metadata at once as one big class
mdata = czimd.CziMetadata(fc.selected)

# get the CZI metadata dictionary directly from filename
mdict = czimd.create_mdict_red(mdata, sort=False)

# convert metadata dictionary to a pandas dataframe
mdframe = misc_tools.md2dataframe(mdict)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(mdframe.to_html()))

,Parameter,Value
0,Directory,F:\Github\czitools\data
1,Filename,CellDivision_T=3_Z=5_CH=2_X=240_Y=170.czi
2,AcqDate,2016-02-12T09:41:02.4915604Z
3,CreationDate,2016-02-12T10:41:02.5655647+01:00
4,UserName,M1SRH
5,SW-App,3.2.0.00001
6,SW-Version,ZEN 3.2 (blue edition)
7,SizeX,240
8,SizeY,170
9,SizeZ,5


In [7]:
# write XML to disk
xmlfile = czimd.writexml(fc.selected)
print("XML File written to:", xmlfile)

XML File written to: F:\Github\czitools\data\CellDivision_T=3_Z=5_CH=2_X=240_Y=170_CZI_MetaData.xml


In [8]:
# get the planetable for the CZI file
pt, csvfile = misc_tools.get_planetable(fc.selected,
                                  norm_time=True,
                                  savetable=True,
                                  separator=',',
                                  index=True)

# and display it nicely as a HTML inside the jupyter notebook
display(HTML(pt.to_html()))
print("PlaneTable CSV File written to:", csvfile)

,Subblock,Scene,Tile,T,Z,C,X[micron],Y[micron],Z[micron],Time[s],xstart,ystart,width,height
0,0,0,0,0,0,0,16977.153,18621.489,1114.49,0.000000,0,0,240,170
1,1,0,0,0,0,1,16977.153,18621.489,1114.49,0.328019,0,0,240,170
2,2,0,0,0,1,0,16977.153,18621.489,1114.81,0.894051,0,0,240,170
3,3,0,0,0,1,1,16977.153,18621.489,1114.81,1.220070,0,0,240,170
4,4,0,0,0,2,0,16977.153,18621.489,1115.13,1.792102,0,0,240,170
5,5,0,0,0,2,1,16977.153,18621.489,1115.13,2.117121,0,0,240,170
6,6,0,0,0,3,0,16977.153,18621.489,1115.45,2.681153,0,0,240,170
7,7,0,0,0,3,1,16977.153,18621.489,1115.45,3.007172,0,0,240,170
8,8,0,0,0,4,0,16977.153,18621.489,1115.77,3.586205,0,0,240,170
9,9,0,0,0,4,1,16977.153,18621.489,1115.77,3.941225,0,0,240,170


PlaneTable CSV File written to: F:\Github\czitools\data\CellDivision_T=3_Z=5_CH=2_X=240_Y=170_planetable.csv
